In [ ]:
import os, re, sys
import subprocess as sp
from glob import glob

## 1 Fundamental settings of the pipeline

In [ ]:
class dress_room(object):
    def __init__(self):
        self.REF = '/home/seonwhee/Bioinformatics/Pipeline/Reference/'
        self.hg38 = self.REF + 'ucsc_hg38/hg38'
        self.FASTQ_LOCATION = '/home/seonwhee/Bioinformatics/IRCR/EQL2/RNA_20170802'
        self.EQL8 = '/home/seonwhee/Bioinformatics/IRCR/EQL8'
        self.BAM_LOCATION = '%s/IRCR_BT16_929_T01_RSq/' %(self.EQL8)
        self.EXPR = '%sOutput/'%(self.BAM_LOCATION)
        
    def command_exec(self, cmd):
        print(' '.join(cmd))
        sp.call(' '.join(cmd), shell=True)
        
        

## 2 Preliminary build of Bowtie index file
#### Since it is an input file of Tophat, index file named hg38.bt2l should be made beforehand.

In [ ]:
class wearing_Bowtie(dress_room, object):
            
    def building_indice(self):
        print(self.hg38)
        cmd = "bowtie2-build"
        arg = " --large-index %s.fa hg38" %(self.hg38)
        try:
            retcode = sp.call(cmd + arg, shell=True)
            if retcode < 0:
                print("Child was terminated by signal", -retcode, file=sys.stderr)
            else:
                print("Child returned", retcode, file=sys.stderr)
        except OSError as e:
            print("Execution failed:", e, file=sys.stderr)

## 3 Mapping against the reference(hg38)

In [ ]:
class wearing_Tophat(dress_room, object):    
        
    def making_alignment(self):
        inputFile1 = '%s/IRCR_BT16_929_T01.RNA.R1.fq.gz'%(self.FASTQ_LOCATION)
        inputFile2 = '%s/IRCR_BT16_929_T01.RNA.R2.fq.gz'%(self.FASTQ_LOCATION)
        CmdArgs = ["tophat2", "-o %s" %(self.BAM_LOCATION), "-p 8", self.hg38, inputFile1, inputFile2]        
        self.command_exec(CmdArgs)           
    
    def check_outputFiles(self):
        import pysam
        BamFile = pysam.AlignmentFile(self.BAM_LOCATION+"accepted_hits.bam", 'rb')
        iter = BamFile.pileup('seq1', 10, 20)
        for x in iter:
            print (str(x))

## 4 Quantification of RNA expression

In [ ]:
class clipping_Cufflinks(dress_room, object):
    def __init__(self):
        super(clipping_Cufflinks, self).__init__()        
    
    def expression_profile(self):
        CmdArgs = ["cufflinks", "-G %s.gtf"%(self.hg38), "-o %s" %(self.EXPR), "%saccepted_hits.bam"%(self.BAM_LOCATION)]
        self.command_exec(CmdArgs)

In [ ]:
class Differential_expression(dress_room, object):
    def __init__(self):
        super(clipping_Cufflinks, self).__init__()
        self.assembly_file = "%s/assembly_list.txt" %(self.EQL8)
        self.merge = self.EQL8 + '/Merging_results'
        self.labels = []
        
    def merging_transcripts(self):        
        assembly_list = glob("%s/Output/transcripts.gtf"%(self.EXPR)) 
        i = 1
        with open(self.assembly_file, 'w') as f:
            for GTF_path in assembly_list:
                f.write("%s\n" %(GTF_path))
                if i == len(assembly_list):
                    self.labels.append(GTF_path.split('/')[-2])
                else:
                    self.labels.append(GTF_path.split('/')[-2]+",")
                i = i+1
        
        with open("%s/label.txt"%(self.merge), 'w') as f:
            label_as_str = ''.join(self.labels)
            f.write(label_as_str)
        CmdArgs = ["cuffmerge", "-g %s.gtf"%(self.hg38), "-s %s.fa"%(self.hg38), "-o %s"%(self.merge), self.assembly_file]
        self.command_exec(CmdArgs)
        
    def differential_expression(self):
        hits_list = glob("%s/*/accepted_hits.bam"%(self.BAM_LOCATION))
        with open("%s/label.txt"%(self.merge), 'r') as f:
            diff_label = f.read()
        CmdArgs = ["cuffdiff", "-q", "-u %s/merged.gtf"%(self.merge), "-N", "-o %s"%(self.merge), "-b %s.fa"%(self.hg38), "-L", diff_label]
        CmdArgs = CmdArgs + hits_list        
        self.command_exec(CmdArgs)
    
    def wearing_cummerbund(self):
        

## 5 Pipeline execution

In [ ]:
ones = wearing_Tophat()
ones.making_alignment()

In [ ]:
ones.check_outputFiles()

In [ ]:
tuxedo = clipping_Cufflinks()
tuxedo.expression_profile()

In [ ]:
diff = Differential_expression()
diff.merging_transcripts()
diff.differential_expression()